## Malaraia Detection using InceptionV3

In [1]:
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import TerminateOnNaN
from tensorflow.keras.callbacks import ProgbarLogger
from tensorflow.keras.callbacks import History
from tensorflow.keras.callbacks import BaseLogger
from tensorflow.keras.callbacks import RemoteMonitor
from tensorflow.keras.callbacks import CallbackList
from tensorflow.keras.callbacks import Callback

In [2]:
image = 'cell_images'

In [3]:
# InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers except the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

# Compile the model
model.compile(loss='binary_crossentropy',
                optimizer=Adam(lr=0.0001),
                metrics=['accuracy'])

<keras.engine.input_layer.InputLayer object at 0x000001B280447190> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001B2978B8310> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001B2978B8970> False
<keras.layers.core.activation.Activation object at 0x000001B2978B8580> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001B2978B8C40> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001B2978BA800> False
<keras.layers.core.activation.Activation object at 0x000001B2978BA4A0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001B2978BB520> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001B2979D87F0> False
<keras.layers.core.activation.Activation object at 0x000001B28006C370> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001B2978BA440> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001B2978B8D

c:\Users\Mostafizur\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [6]:
# Data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# Data augmentation
val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
# Data augmentation
train_generator = train_datagen.flow_from_directory(
        'cell_images',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 27558 images belonging to 2 classes.


In [11]:
test_generator = test_datagen.flow_from_directory(
        'cell_images',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 27558 images belonging to 2 classes.


In [12]:
val_generator = val_datagen.flow_from_directory(
        'Pneumonia',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 5216 images belonging to 2 classes.


In [14]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=True,
                                mode='auto')

In [16]:
# Callbacks
earlystop = EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=5,
                            verbose=0,
                            mode='auto')

In [17]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                factor=0.2,
                                patience=3,
                                verbose=0,
                                mode='auto')

In [21]:
csv_logger = CSVLogger('log.csv',
                        append=False,
                        separator=',')

In [24]:
# Callbacks
tensorboard = TensorBoard(log_dir='logs',
                            histogram_freq=0,
                            write_graph=True,
                            write_images=True)

In [22]:

# Callbacks
lambda_callback = LambdaCallback(on_epoch_begin=None,
                                    on_epoch_end=None,
                                    on_batch_begin=None,
                                    on_batch_end=None,
                                    on_train_begin=None,
                                    on_train_end=None)

In [20]:
terminate_on_nan = TerminateOnNaN()

In [25]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=5216 // 32,
        epochs=100,
        validation_data=val_generator,
        validation_steps=624 // 32,
        callbacks=[checkpoint, earlystop, reduce_lr, tensorboard, csv_logger, lambda_callback, terminate_on_nan])

C:\Users\Mostafizur\AppData\Local\Temp\ipykernel_4728\1784942521.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/100
 32/163 [====>.........................] - ETA: 3:23 - loss: 1.0841 - accuracy: 0.6748

In [ ]:
#Plot the accuracy and loss curves for training and validation
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
#Accuracy
print('Accuracy: ', score[1]*100)
#Loss
print('Loss: ', score[0])